In [2]:
#imports
import pandas as pd
from pathlib import Path

In [ ]:
# load files
new = pd.read_excel("RES25_new.xlsx", sheet_name="Main")
prev = pd.read_excel("Previous.xlsx", sheet_name="Main")
archived = pd.read_excel("ETO_Archived_Resource_List.xlsx")
portfolio = pd.read_excel("ETO_Resource_Portfolio.xlsx")

In [ ]:
# 1) convert anonymized using name or WorkdayID
# assume 'WorkdayID' column and 'AnonymizedName' mapping exists in archived
new = new.merge(archived[['AnonymizedName','WorkdayID','RealName']], how='left', left_on='ResourceName', right_on='AnonymizedName')
new['WorkdayID'] = new['WorkdayID'].fillna(new['WorkdayID_x']).fillna(new['WorkdayID_y'])

In [ ]:
# 2) resource type from previous
prev_types = prev[['WorkdayID','ResourceType']].drop_duplicates()
new = new.merge(prev_types, on='WorkdayID', how='left', suffixes=('','_prev'))
new['ResourceType'] = new['ResourceType'].fillna(new['ResourceType_prev'])

In [ ]:
# 3) Category mapping example (needs 'TS Status', 'Effort Type', current week col name)
def compute_category(row, current_week_col):
    if row['TS Status'] == 'Progressed':
        return 'Actuals'
    if row['TS Status'] == 'N/A':
        return 'Forecast'
    if row['Effort Type'] == 'Reported':
        if row[current_week_col] != 0.0:
            return 'Reported-IP'
        else:
            return 'Reported-MO'
    return ''

new['Category'] = new.apply(lambda r: compute_category(r, 'FY2026_WK12'), axis=1)

In [ ]:
# 4) Zero out weeks depending on Category (example)
week_cols = [c for c in new.columns if c.startswith('FY')]
for c in week_cols:
    new.loc[(new['Category']=='Actuals') & (pd.to_datetime(c) > pd.Timestamp.today()), c] = 0.0
    # ...similar rules for Forecast / Reported-IP / Reported-MO

In [ ]:
# 5) Recalc totals & cost
new['TotalHours_new'] = new[week_cols].sum(axis=1)
# derive hourly rate
new['hourly_rate'] = new.apply(lambda r: r['CostUSD']/r['Total (Hours)'] if r['Total (Hours)']>0 else None, axis=1)
new['CostUSD_new'] = new['hourly_rate'] * new['TotalHours_new']